In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser


## Simple Chain
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}. Respond in {language}"
)

model = AzureChatOpenAI(temperature=0.0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

chain = prompt | model
response = chain.invoke({"topic": "toddlers", "language": "telugu"})
response

AIMessage(content='ఒక తల్లి: నా కూతురి చాలా చిన్నాడు, కానీ అతను చాలా మాటలు మాట్లాడుతుంది.\nమిత్రుడు: అవ్వాలి, అందుకే తల్లిని అందరికీ అర్థం కాకుండా మాటలు మాట్లాడుతుంది.\nతల్లి: అయ్యో! అందుకే నాకు అర్థం కాకుండా మాటలు మాట్లాడుతుంది.')

In [4]:
# Attaching kwargs that will be passed to each model call
chain = prompt | model.bind(stop=["\n"])
response = chain.invoke({"topic": "toddlers"})
response

AIMessage(content='Why did the toddler bring a ladder to the playground?')

In [3]:
output_parser = StrOutputParser()
chain = prompt | model | output_parser
response = chain.invoke({"topic": "toddlers"})
print(response)

Why don't toddlers ever win at poker?

Because they always struggle to keep a straight face!


In [8]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes? \n\nBecause they have bear feet!",
 'Why did the frog bring a suitcase to the party?\n\nBecause he wanted to "hop" on a plane!']

In [10]:
for t in chain.stream({"topic": "kids"}):
    print(t)


Why
 don
't
 scientists
 trust
 atoms
?
 


Because
 they
 make
 up
 everything
!



In [11]:
response = await chain.ainvoke({"topic": "infants"})
response

'Why don\'t infants ever gamble? \n\nBecause they\'re always a little too "crib" shy!'

Functions

In [6]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)
response = chain.invoke({"topic": "toddlers"}, config={})
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t toddlers ever listen to classical music?",\n  "punchline": "Because they prefer to conduct their own tantrums!"\n}'}})

In [7]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)
response = chain.invoke({"topic": "toddlers"}, config={})
response

{'setup': "Why don't toddlers ever tell secrets?",
 'punchline': "Because they can't keep their mouths shut!"}

In [8]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)
response = chain.invoke({"topic": "toddlers"}, config={})
response

"Why don't toddlers ever get jokes?"

In [11]:
# Simplifying Input

from langchain.schema.runnable import RunnableMap, RunnablePassthrough

map_ = RunnableMap(topic=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)
chain.invoke("toddlers")

{'setup': "Why don't toddlers ever get jokes?",
 'punchline': "Because they can't understand the punchline!"}

In [13]:
# Syntactic Sugar

chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)
chain.invoke("toddlers")

"Why don't toddlers ever win races?"